In [323]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [324]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [325]:
### Load NER model
model_ner = spacy.load('./output/model_web_lg/model-best/')

In [326]:
# Load image
image = cv2.imread('./data/6.jpg')

# cv2.imshow('business card', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [327]:
# extract data using Pytesseract
textData = pytesseract.image_to_data(image)
tess_list = list(map(lambda x:x.split('\t'),textData.split('\n')))
df = pd.DataFrame(tess_list[1:], columns=tess_list[0])
df.dropna(inplace=True) # drop missing values
df['text'] = df['text'].apply(cleanText)

In [328]:
# convert data into context
df_clean = df.query("text != '' ")
content = " ".join([w for w in df_clean['text']])
df_clean

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
12,5,1,3,1,1,1,722,53,64,28,90.167198,cell
14,5,1,3,1,1,3,822,53,203,28,96.483879,8099948528
17,5,1,3,2,1,1,55,55,85,89,47.185547,ga
18,5,1,3,2,1,2,822,95,203,28,96.435081,8466045457
20,5,1,3,2,2,1,593,136,93,25,89.289780,email
22,5,1,3,2,2,3,709,136,316,31,86.249245,lictsrikant@gmail.com
25,5,1,3,3,1,1,46,170,33,14,96.406654,life
26,5,1,3,3,1,2,85,151,92,42,95.806709,insurance
27,5,1,3,3,1,3,183,170,117,14,96.909729,corporation
28,5,1,3,3,1,4,306,170,20,14,96.172005,of


In [329]:
# get predictions from NER model
doc = model_ner(content)
doc

cell 8099948528 ga 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail “com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo

In [330]:
from spacy import displacy
# displacy.serve(doc, style='ent')

In [331]:
displacy.render(doc, style='ent')

## Tagging

In [332]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [333]:
doc_text = docjson['text']

In [334]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start', 'end']].apply(
    lambda x: doc_text[x[0]: x[1]], axis=1)


C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\4153108972.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: doc_text[x[0]: x[1]], axis=1)


In [335]:
datafram_tokens.head(25)

,id,start,end,token
0,0,0,4,cell
1,1,5,15,8099948528
2,2,16,18,ga
3,3,19,29,8466045457
4,4,30,35,email
5,5,36,57,lictsrikant@gmail.com
6,6,58,62,life
7,7,63,72,insurance
8,8,73,84,corporation
9,9,85,87,of


In [336]:
right_table = pd.DataFrame(docjson['ents'])[['start', 'label']]
datafram_tokens = pd.merge(datafram_tokens, right_table, how='left', on='start')

In [337]:
datafram_tokens

,id,start,end,token,label
0,0,0,4,cell,NaN
1,1,5,15,8099948528,B-PHONE
2,2,16,18,ga,I-PHONE
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,NaN
...,...,...,...,...,...
58,58,448,455,promote,NaN
59,59,456,460,your,NaN
60,60,461,469,business,NaN
61,61,470,476,online,B-ORG


In [338]:
datafram_tokens.fillna('O', inplace=True)
datafram_tokens.head(25)

,id,start,end,token,label
0,0,0,4,cell,O
1,1,5,15,8099948528,B-PHONE
2,2,16,18,ga,I-PHONE
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,O
5,5,36,57,lictsrikant@gmail.com,B-EMAIL
6,6,58,62,life,B-ORG
7,7,63,72,insurance,I-ORG
8,8,73,84,corporation,I-ORG
9,9,85,87,of,I-ORG


In [339]:
# Join Lable to df_clean dataframe
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 # Correct end position
df_clean['start'] = df_clean[['text', 'end']].apply(lambda x: x[1] - len(x[0]), axis=1)

C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\2501512021.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 # Correct end position
C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\2501512021.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_clean['start'] = df_clean[['text', 'end']].apply(lambda x: x[1] - len(x[0]), axis=1)
C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\2501512021.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [340]:
# inner join with start
dataframe_info = pd.merge(df_clean, datafram_tokens[['start', 'token', 'label']], how='inner', on='start')

## Bounding box

In [341]:
bb_df = dataframe_info.query("label != 'O'")
img = image.copy()

for x, y, w, h, label in bb_df[['left', 'top', 'width', 'height', 'label']].values:
    x, y, w, h = int(x), int(y), int(w), int(h)

    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(img, str(label), (x, y), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0),2)

cv2.imshow('Predictions', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [342]:
print(bb_df)

   level page_num block_num par_num line_num word_num left  top width height  \
1      5        1         3       1        1        3  822   53   203     28   
2      5        1         3       2        1        1   55   55    85     89   
3      5        1         3       2        1        2  822   95   203     28   
5      5        1         3       2        2        3  709  136   316     31   
6      5        1         3       3        1        1   46  170    33     14   
7      5        1         3       3        1        2   85  151    92     42   
8      5        1         3       3        1        3  183  170   117     14   
9      5        1         3       3        1        4  306  170    20     14   
10     5        1         3       3        1        5  332  170    42     14   
11     5        1         3       3        1        6  668  166   119     37   
13     5        1         3       3        1        8  958  180    67     19   
14     5        1         4       1     

In [343]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\2854881843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,3,1,1,3,822,53,203,28,96.483879,8099948528,15,5,8099948528,PHONE
2,5,1,3,2,1,1,55,55,85,89,47.185547,ga,18,16,ga,PHONE
3,5,1,3,2,1,2,822,95,203,28,96.435081,8466045457,29,19,8466045457,PHONE
5,5,1,3,2,2,3,709,136,316,31,86.249245,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL
6,5,1,3,3,1,1,46,170,33,14,96.406654,life,62,58,life,ORG


In [344]:
# group the Label
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''

    def getgroup(self, text):
        if self.text == text:
            return self.id
        else:
            self.id += 1
            self.text = text
            return self.id

grp_gen = groupgen()

In [345]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)

C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\472836701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)


In [346]:
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label,group
1,5,1,3,1,1,3,822,53,203,28,96.483879,8099948528,15,5,8099948528,PHONE,1
2,5,1,3,2,1,1,55,55,85,89,47.185547,ga,18,16,ga,PHONE,1
3,5,1,3,2,1,2,822,95,203,28,96.435081,8466045457,29,19,8466045457,PHONE,1
5,5,1,3,2,2,3,709,136,316,31,86.249245,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL,2
6,5,1,3,3,1,1,46,170,33,14,96.406654,life,62,58,life,ORG,3
7,5,1,3,3,1,2,85,151,92,42,95.806709,insurance,72,63,insurance,ORG,3
8,5,1,3,3,1,3,183,170,117,14,96.909729,corporation,84,73,corporation,ORG,3
9,5,1,3,3,1,4,306,170,20,14,96.172005,of,87,85,of,ORG,3
10,5,1,3,3,1,5,332,170,42,14,96.796776,india,93,88,india,ORG,3
11,5,1,3,3,1,6,668,166,119,37,92.970848,seosrika,102,94,seosrika,NAME,4


In [347]:
# right and botton of bouding box
# right and bottom of bounding box
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\1852051330.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\1852051330.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['right'] = bb_df['left'] + bb_df['width']
C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\1852051330.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [348]:
# tagging: groupby group
col_group = ['left','top','right','bottom','label','token','group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [349]:
img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'token':lambda x: " ".join(x)
    
})

C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\976584219.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  img_tagging = group_tag_img.agg({
C:\Users\Alex\AppData\Local\Temp\ipykernel_31832\976584219.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  img_tagging = group_tag_img.agg({


In [350]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,55,1025,53,144,[PHONE],8099948528 ga 8466045457
2,709,1025,136,167,[EMAIL],lictsrikant@gmail.com
3,46,374,151,193,[ORG],life insurance corporation of india
4,668,787,166,203,[NAME],seosrika
5,958,1025,180,199,[WEB],“
6,310,755,227,259,[NAME],thathineni srikanth
7,399,669,271,296,[DES],insurance advisor
8,47,882,310,592,[ORG],0316164y life insurance corporation of india o...
9,432,528,571,598,[NAME],pybo


In [351]:
img_bb = image.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb,(l,t),(r,b),(0,255,0),2)
    
    cv2.putText(img_bb,str(label),(l,t),cv2.FONT_HERSHEY_PLAIN,1,(0,0,0),2)
    
    
cv2.imshow('Bounding Box BusinessCard',img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Parser

In [352]:
def parser(text, label):
    if label == 'PHONE':
        text = text.lower()
        text = re.sub(r'\D', '', text)

    elif label == 'EMAIL':
        text = text.lower()
        allow_special_char = '@_.\\-'
        text = re.sub(r'[^A-Za-z0-9{}]'.format(allow_special_char), '', text)

    elif label == 'WEB' :
        text = text.lower()
        allow_special_char = ':/.%#\\-'
        text = re.sub(r'[^A-Za-z0-9{}]'.format(allow_special_char), '', text)

    elif label in ('NAME', 'DES'):
        text = text.lower()
        text = re.sub(r'[^a-z]', '', text)
        text = text.title()

    elif label == 'ORG':
        text = text.lower()
        text = re.sub(r'[^a-z0-9]', '', text)
        text = text.title()

    return text

## Entities

In [353]:
info_array = dataframe_info[['token', 'label']].values
entities = dict(NAME=[], ORG=[], DES=[], PHONE=[], EMAIL=[], WEB=[])
previous = 'O'

for token, label in info_array:
    bio_tag = label[0]
    label_tag = label[2:]

    # step 1 - parse the token
    text = parser(token, label_tag)

    if bio_tag in ('B', 'I'):
        if previous != label_tag:
            entities[label_tag].append(text)

        else:
            if bio_tag == 'B':
                entities[label_tag].append(text)

            else:
                if label_tag in ("NAME", "ORG", "DES"):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text

                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + text
    previous = label_tag


In [354]:
entities

{'NAME': ['Seosrika', 'Thathineni Srikanth', 'Pybo'],
 'ORG': ['Life Insurance Corporation Of India',
  '0316164Y',
  'Life Insurance Corporation Of India',
  'Online'],
 'DES': ['Insurance Advisor'],
 'PHONE': ['8099948528', '8466045457'],
 'EMAIL': ['lictsrikant@gmail.com'],
 'WEB': ['']}